<center> <h1> 600092 <br>  Data Mining and Decision Systems </h1> </center>

---

<center> <h2> Data Cleaning </h2> </center>

#### Importing necessary libraries.

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import scikitplot as skplt

#### Importing dataset and creating a copy of the dataset.

In [2]:
non_standard_vals = ["na", "n/a", "--", " "]
df = pd.read_csv('600092_data.csv', na_values=non_standard_vals)

df_copy = df.copy()

Also here, using non_standard_vals, this checks if there's any non-standard null values such as `na, n/a, --, " "` in the dataset, then in the import, those values are replaced with correct standard null values.

 #### Making sure the data has been correctly imported

In [3]:
df_copy.head() # Grabs the first 5 entities

Random      Id Indication Diabetes  IHD Hypertension Arrhythmia History  \
0  0.602437  218242        A-F       no   no          yes         no      no   
1  0.602437  159284        TIA       no   no           no         no      no   
2  0.602437  106066        A-F       no  yes          yes         no      no   
3  0.128157  229592        TIA       no   no          yes         no      no   
4  0.676862  245829        CVA       no   no           no         no      no   

   IPSI  Contra   label  
0  78.0    20.0  NoRisk  
1  70.0    60.0  NoRisk  
2  95.0    40.0    Risk  
3  90.0    85.0    Risk  
4  70.0    20.0  NoRisk

Looks like the data has imported correctly, now data cleaning can begin.

#### Understanding the dataset


In [4]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1520 entries, 0 to 1519
Data columns (total 11 columns):
Random          1520 non-null float64
Id              1520 non-null int64
Indication      1517 non-null object
Diabetes        1518 non-null object
IHD             1520 non-null object
Hypertension    1517 non-null object
Arrhythmia      1520 non-null object
History         1518 non-null object
IPSI            1516 non-null float64
Contra          1519 non-null float64
label           1517 non-null object
dtypes: float64(3), int64(1), object(7)
memory usage: 130.8+ KB


Using `df_copy.info()` to briefly summarise the dataset, to better understand the data.

This shows how many data entities there are, and the different data types.

#### Checking for null values

This is just a check whether there is any null values 

In [5]:
df_copy.isnull().values.any() 

True

This returns a boolean value, and as shown, it is `true` there is null values


**Where are the null values?**

In [6]:
df_copy.isnull().sum()

Random          0
Id              0
Indication      3
Diabetes        2
IHD             0
Hypertension    3
Arrhythmia      0
History         2
IPSI            4
Contra          1
label           3
dtype: int64

This shows the amount of null values each column has, therefore as decision could be made to either fill the values in or drop them entirely. 

As shown there is 18 NaN values.

#### Percentage of Null values 

In [7]:
null_val_percent = df_copy.isnull().sum()/len(df_copy)*100

null_val_percent

Random          0.000000
Id              0.000000
Indication      0.197368
Diabetes        0.131579
IHD             0.000000
Hypertension    0.197368
Arrhythmia      0.000000
History         0.131579
IPSI            0.263158
Contra          0.065789
label           0.197368
dtype: float64

As seen there is very similar percentage of null values distributed within the data set 

#### Context to null values

#### Visualisation of Null value placements

In [8]:
sns.heatmap(null_val_percent.to_frame())

In [9]:
display_nulls = df_copy[df_copy.isna().any(axis=1)]

display_nulls

Random      Id Indication Diabetes  IHD Hypertension Arrhythmia  \
201   0.388790  240232        CVA      yes   no          NaN        yes   
337   0.617072  260770        CVA       no  yes           no         no   
442   0.648122  261060        NaN       no  yes          yes         no   
447   0.681744  224257        CVA      NaN  yes          yes        yes   
453   0.518558  254469        ASx       no   no          yes         no   
514   0.392130  210861        A-F      NaN   no          yes         no   
590   0.707950  289096        A-F       no   no          NaN         no   
620   0.283872  227779        A-F       no   no           no         no   
752   0.243518  146415        CVA       no  yes          yes        yes   
784   0.243645  196483        CVA       no   no          yes        yes   
812   0.756938  190671        CVA       no  yes          yes         no   
822   0.211760  205845        NaN       no  yes           no         no   
845   0.920096  104176        CVA       no   no           no         no   
916   0.004501  281679        A-F       no   no           no         no   
974   0.923418  165432        CVA       no  yes          yes        yes   
983   0.319215  158024        TIA       no   no          NaN         no   
1007  0.761219  256076        NaN       no   no           no        yes   
1418  0.813102  169726        TIA      yes   no           no         no   

     History  IPSI  Contra   label  
201       no  95.0    90.0    Risk  
337      NaN  85.0    50.0  NoRisk  
442       no  70.0    40.0  NoRisk  
447       no  90.0   100.0    Risk  
453       no  70.0     NaN  NoRisk  
514       no  80.0    40.0  NoRisk  
590       no  78.0    30.0  NoRisk  
620       no   NaN   100.0    Risk  
752       no   NaN   100.0    Risk  
784      NaN  95.0   100.0    Risk  
812       no  76.0    50.0     NaN  
822       no  80.0   100.0    Risk  
845       no   NaN    20.0  NoRisk  
916       no   NaN    50.0  NoRisk  
974       no  80.0    70.0     NaN  
983       no  75.0    75.0  NoRisk  
1007     yes  99.0   100.0    Risk  
1418     yes  90.0    50.0     NaN

This shows all the null values within the data, this makes it useful to give context where the null values are relation to the data. 

**Now a decision can be made either to drop the values or fill them in.**

As there is only 18 data entities that contain `NaN` values, filling in the data values with either a `fillna(ffill)` or `fillna(bfill` may skew visualisation of the results or any models that will be made in the future, possibly incorrectly training the models. 

Along with this, the context of the data is whether a patient is at risk or not of death regarding all the possible health issues they may or may not have, which either a forward fill or backwards fill could incorrectly label them which would could cause issues later down the line, especially with training a model. 

Therefore, it will be better practise to drop the `NaN` values.

Here the heat map shows a distribution of null values in the columns

In [10]:
df_copy.dropna(inplace=True)

**Checking if `NaN` values have been dropped**

In [11]:
df_copy.isnull().sum()

Random          0
Id              0
Indication      0
Diabetes        0
IHD             0
Hypertension    0
Arrhythmia      0
History         0
IPSI            0
Contra          0
label           0
dtype: int64

As we can see all the `NaN` values have been dropped, and progress can be made. 

#### Correcting Data types

According to the briefing, not all these data types are correct. Such as `Contra` and `IPSI`, which must be converted into integers respectively.

First, its best to check the data types:

In [12]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1502 entries, 0 to 1519
Data columns (total 11 columns):
Random          1502 non-null float64
Id              1502 non-null int64
Indication      1502 non-null object
Diabetes        1502 non-null object
IHD             1502 non-null object
Hypertension    1502 non-null object
Arrhythmia      1502 non-null object
History         1502 non-null object
IPSI            1502 non-null float64
Contra          1502 non-null float64
label           1502 non-null object
dtypes: float64(3), int64(1), object(7)
memory usage: 140.8+ KB


It's not necessary for `IPSI` and `Contra` to be `Float64`, they are much better suited to be `Int64` instead.

In [13]:
df_copy["IPSI"] = df_copy["IPSI"].astype(np.int64)

In [14]:
df_copy["Contra"] = df_copy["Contra"].astype(np.int64)

Now to check if the conversion worked.

In [15]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1502 entries, 0 to 1519
Data columns (total 11 columns):
Random          1502 non-null float64
Id              1502 non-null int64
Indication      1502 non-null object
Diabetes        1502 non-null object
IHD             1502 non-null object
Hypertension    1502 non-null object
Arrhythmia      1502 non-null object
History         1502 non-null object
IPSI            1502 non-null int64
Contra          1502 non-null int64
label           1502 non-null object
dtypes: float64(1), int64(3), object(7)
memory usage: 140.8+ KB


As shown the conversion worked.

#### Checking for incorrect values

It's necessary to check whether there is valid data in the dataset, spelling mistakes, unknow values or incorrect datatypes can cause issues with further data manipulation.

**Checking columns for incorrect values**

In [16]:
for c in df_copy.columns: 
        print(df_copy[c].name)
        print(df_copy[c].value_counts(), ("\n"))

Random
0.683506    4
0.506597    4
0.576184    4
0.899742    4
0.803415    3
           ..
0.801775    1
0.972183    1
0.692722    1
0.675468    1
0.219026    1
Name: Random, Length: 1211, dtype: int64 

Id
280575    1
260803    1
174778    1
170683    1
141608    1
         ..
157020    1
259831    1
163168    1
215064    1
221184    1
Name: Id, Length: 1502, dtype: int64 

Indication
A-F    489
CVA    408
TIA    388
ASx    185
Asx     32
Name: Indication, dtype: int64 

Diabetes
no     1427
yes      75
Name: Diabetes, dtype: int64 

IHD
no     790
yes    712
Name: IHD, dtype: int64 

Hypertension
no     777
yes    725
Name: Hypertension, dtype: int64 

Arrhythmia
no     1178
yes     324
Name: Arrhythmia, dtype: int64 

History
no     1480
yes      22
Name: History, dtype: int64 

IPSI
70    229
75    221
80    185
90    173
95    155
73     83
85     76
76     74
78     49
68     32
99     30
60     29
81     28
82     25
50     23
65     14
79     13
71     13
55      9
77      9
74

As seen above, in the column `Indication` there is a possible spelling mistake, as there's `Asx` with 32 values, instead of `ASx` which has 185 values. As `ASx` has a high amount of values, its safe to presume that `Asx` is a miss-spelling and should be rectified. 


Also, in the column `label`, there is 2 `unknown` values. Which should be viewed and then either fixed or dropped.

In [17]:
df_copy["Indication"] = df_copy["Indication"].replace({'Asx':'ASx'})

This replaces the miss-spelling with the correct spelling.

Then to check if this has worked

In [18]:
df_copy["Indication"].value_counts()

A-F    489
CVA    408
TIA    388
ASx    217
Name: Indication, dtype: int64

Which is has, now the incorrect spelling has been rectifed.

**Moving onto the unknown values in `label`** 

In [19]:
df_copy.loc[df_copy["label"] == "Unknown"]

Random      Id Indication Diabetes  IHD Hypertension Arrhythmia  \
475  0.298074  173791        ASx       no  yes          yes         no   
523  0.460170  283846        CVA       no   no          yes        yes   

    History  IPSI  Contra    label  
475      no    70      55  Unknown  
523      no    95     100  Unknown

This shows that `Unknown` is a string value, therefore can just be dropped, as it provides incorrect data. 

In [20]:
unknown_index = df_copy[df_copy["label"] == "Unknown" ].index

df_copy.drop(unknown_index , inplace=True)

Now to check this

In [21]:
df_copy["label"].value_counts()

NoRisk    1000
Risk       500
Name: label, dtype: int64

As seen, there is no `Unknown` value any more

#### Looking for duplicate values

It's nescessary to find incorrect duplicate values, the only column that shouldn't have duplicated values is `Random`, we can also check if `Id` contains duplicates as the column allows them.

In [22]:
df_copy["Random"].duplicated().any()

True

So there is duplicated values in `Random`

In [23]:
df_copy["Id"].duplicated().values.any()

False

And there are no duplicates in `Id`, this suggests that there was a error in naming the columns, or just incorrect data input. 

In [24]:
float(amount_of_duplicates) = df_copy["Random"].duplicated().sum() 

amount_of_duplicates

289.0

#### Visualising duplicates in `Random` column

In [38]:
amount_of_unique = float(len(df) - amount_of_duplicates)

labels = 'Duplicated Values', 'Unique Values'
sizes = [amount_of_duplicates,amount_of_unique.float]
colors = ['#66b3ff','#99ff99']

plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%',
        shadow=True, startangle=90)

plt.axis("equal")
plt.show()

AttributeError: 'float' object has no attribute 'float'

Here we can see the distribution of duplicates in the `Random` column

This shows a quite substaination amount of duplicates. There are two options, either just drop the column or drop the duplicate rows. 

As there could be model training with this dataset it makes more sense just to drop the column.

In [26]:
df_copy.drop("Random", axis=1, inplace = True)

Now to check if the drop has worked.

In [27]:
df_copy.head()

Id Indication Diabetes  IHD Hypertension Arrhythmia History  IPSI  \
0  218242        A-F       no   no          yes         no      no    78   
1  159284        TIA       no   no           no         no      no    70   
2  106066        A-F       no  yes          yes         no      no    95   
3  229592        TIA       no   no          yes         no      no    90   
4  245829        CVA       no   no           no         no      no    70   

   Contra   label  
0      20  NoRisk  
1      60  NoRisk  
2      40    Risk  
3      85    Risk  
4      20  NoRisk

As seen, it has worked fine.

#### Unnecessary columns

Some of the columns are unnecessary for data analysis. One of these columns is `Id`, as it is unnecessary for an identifier for data analysis. 

The other identifier `Random` has already been dropped, so the only other identifer is `Id` as its not necessary, it can be dropped also. 

In [28]:
df_copy.drop("Id", axis=1, inplace = True)

Now to check if the drop has worked

In [29]:
df_copy.head()

Indication Diabetes  IHD Hypertension Arrhythmia History  IPSI  Contra  \
0        A-F       no   no          yes         no      no    78      20   
1        TIA       no   no           no         no      no    70      60   
2        A-F       no  yes          yes         no      no    95      40   
3        TIA       no   no          yes         no      no    90      85   
4        CVA       no   no           no         no      no    70      20   

    label  
0  NoRisk  
1  NoRisk  
2    Risk  
3    Risk  
4  NoRisk

Which it has.

#### Mapping Columns to Bools

In [30]:
df_copy["Indication"] = df_copy["Indication"].map({"A-F": "0", "TIA":"1", "CVA": "2", "ASx": "3"})
df_copy["Diabetes"] = df_copy["Diabetes"].map({"no":"0", "yes":"1"})
df_copy["IHD"] = df_copy["IHD"].map({"no":"0", "yes":"1"})
df_copy["Hypertension"] = df_copy["Hypertension"].map({"no":"0", "yes":"1"})
df_copy["Arrhythmia"] = df_copy["Arrhythmia"].map({"no":"0", "yes":"1"})
df_copy["History"] = df_copy["History"].map({"no":"0", "yes":"1"})
df_copy["label"] = df_copy["label"].map({"NoRisk":"0", "Risk":"1"})

In [31]:
df_copy["label"]

0       0
1       0
2       1
3       1
4       0
       ..
1515    0
1516    1
1517    0
1518    0
1519    0
Name: label, Length: 1500, dtype: object

### Data Modeling 